In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%%time
from influxdb import InfluxDBClient
from  credentials import *
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
import plotly.graph_objs as go

#import plotly.graph_objs as go
#from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler

import os
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import LSTM, Dense




CPU times: user 649 ms, sys: 158 ms, total: 808 ms
Wall time: 1.02 s


In [4]:
#connect to Database
def get_connect():
    client = InfluxDBClient(host=host, port=port, username=username, password=password)
    client.switch_database('orkney')
    return client


#Query Data
def get_Demand_data():
    client = get_connect()
    Demand = client.query('SELECT * FROM "Demand" WHERE time > now() - 60d ORDER BY time')
    values = Demand.raw["series"][0]["values"]
    columns = Demand.raw["series"][0]["columns"]
    Demand = pd.DataFrame(values, columns=columns)#.set_index("time")
    return Demand



def explore_data(df):
    #df = get_Demand_data()
    df['Datetime'] = pd.to_datetime(df.time)
    df['year'] = df.Datetime.dt.year
    df['month'] = df.Datetime.dt.month
    df['day']=  df.Datetime.dt.day
    df['Hour']= df.Datetime.dt.hour
    df = df.drop(["time","year","month","day","Hour"],axis=1)
    df = df.set_index("Datetime")

    return df 



df = get_Demand_data()
df = explore_data(df)
df.head(2)

Total
Datetime                        
2022-01-08 16:48:00+00:00  23.28
2022-01-08 16:49:00+00:00  23.28

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 79375 entries, 2022-01-08 16:48:00+00:00 to 2022-03-09 16:47:00+00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Total   79375 non-null  float64
dtypes: float64(1)
memory usage: 1.2 MB


In [6]:
##################################
%time
if not os.path.exists('model'):
    os.makedirs('model')

if not os.path.exists('img'):
    os.makedirs('img')

# Hardcode all variables
batch_size_exp = 1
epoch_exp = 1
neurons_exp = 10
predict_values_exp = 60*24*30 #next minute
lag_exp=5

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


In [7]:
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = pd.DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = pd.concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df



In [8]:
df.values

array([[23.28],
       [23.28],
       [23.28],
       ...,
       [20.7 ],
       [21.25],
       [21.25]])

In [9]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)


In [10]:
# transform data to be stationary
raw_values = df.values
diff_values = difference(raw_values, 1)
diff_values[:16] #every 5. step

0                     [0.0]
1                     [0.0]
2                     [0.0]
3                     [0.0]
4                     [0.0]
5                     [0.0]
6      [-0.240000000000002]
7                     [0.0]
8                     [0.0]
9                     [0.0]
10                    [0.0]
11                    [0.0]
12                    [0.0]
13    [-0.2699999999999996]
14                    [0.0]
15                    [0.0]
dtype: object

In [11]:
# transform data to be supervised learning
lag_exp=5
supervised = timeseries_to_supervised(diff_values, lag_exp)

supervised[:10]

0                     0                     0      0  \
0                     0                     0                     0      0   
1                 [0.0]                     0                     0      0   
2                 [0.0]                 [0.0]                     0      0   
3                 [0.0]                 [0.0]                 [0.0]      0   
4                 [0.0]                 [0.0]                 [0.0]  [0.0]   
5                 [0.0]                 [0.0]                 [0.0]  [0.0]   
6                 [0.0]                 [0.0]                 [0.0]  [0.0]   
7  [-0.240000000000002]                 [0.0]                 [0.0]  [0.0]   
8                 [0.0]  [-0.240000000000002]                 [0.0]  [0.0]   
9                 [0.0]                 [0.0]  [-0.240000000000002]  [0.0]   

       0                     0  
0      0                 [0.0]  
1      0                 [0.0]  
2      0                 [0.0]  
3      0                 [0.0]  
4      0                 [0.0]  
5  [0.0]                 [0.0]  
6  [0.0]  [-0.240000000000002]  
7  [0.0]                 [0.0]  
8  [0.0]                 [0.0]  
9  [0.0]                 [0.0]

In [12]:
supervised_values = supervised.values
supervised_values[:6]

array([[0, 0, 0, 0, 0, array([0.])],
       [array([0.]), 0, 0, 0, 0, array([0.])],
       [array([0.]), array([0.]), 0, 0, 0, array([0.])],
       [array([0.]), array([0.]), array([0.]), 0, 0, array([0.])],
       [array([0.]), array([0.]), array([0.]), array([0.]), 0,
        array([0.])],
       [array([0.]), array([0.]), array([0.]), array([0.]), array([0.]),
        array([0.])]], dtype=object)

In [13]:
# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]


In [14]:
# split data into train and test-sets

predict_values_exp = 60 #next predicting minute
train, test = supervised_values[0:-predict_values_exp], supervised_values[-predict_values_exp:]
print(test[:4])

[[array([0.]) array([0.]) array([0.]) array([0.]) array([-0.42])
  array([0.])]
 [array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])]
 [array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
  array([0.56])]
 [array([0.56]) array([0.]) array([0.]) array([0.]) array([0.])
  array([0.])]]


In [15]:
print(train[:6])

[[0 0 0 0 0 array([0.])]
 [array([0.]) 0 0 0 0 array([0.])]
 [array([0.]) array([0.]) 0 0 0 array([0.])]
 [array([0.]) array([0.]) array([0.]) 0 0 array([0.])]
 [array([0.]) array([0.]) array([0.]) array([0.]) 0 array([0.])]
 [array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])]]


In [16]:
# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

In [17]:
# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)


In [18]:
print(scaler)

MinMaxScaler(copy=True, feature_range=(-1, 1))


In [19]:
train[:2]

array([[0, 0, 0, 0, 0, array([0.])],
       [array([0.]), 0, 0, 0, 0, array([0.])]], dtype=object)

In [20]:
print(train_scaled[:2])

[[-0.02768549 -0.02768549 -0.02768549 -0.02768549 -0.02768549 -0.02768549]
 [-0.02768549 -0.02768549 -0.02768549 -0.02768549 -0.02768549 -0.02768549]]


In [21]:
test[:2]

array([[array([0.]), array([0.]), array([0.]), array([0.]),
        array([-0.42]), array([0.])],
       [array([0.]), array([0.]), array([0.]), array([0.]), array([0.]),
        array([0.])]], dtype=object)

In [22]:
print(test_scaled[:2])

[[-0.02768549 -0.02768549 -0.02768549 -0.02768549 -0.12070875 -0.02768549]
 [-0.02768549 -0.02768549 -0.02768549 -0.02768549 -0.02768549 -0.02768549]]


In [23]:
# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]


In [24]:
X, y = train[:, 0:-1], train[:, -1]
print(X[:6])

[[0 0 0 0 0]
 [array([0.]) 0 0 0 0]
 [array([0.]) array([0.]) 0 0 0]
 [array([0.]) array([0.]) array([0.]) 0 0]
 [array([0.]) array([0.]) array([0.]) array([0.]) 0]
 [array([0.]) array([0.]) array([0.]) array([0.]) array([0.])]]


In [25]:
X.shape

(79314, 5)

In [26]:
y.shape

(79314,)

In [27]:
print(y[:10])

[array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([-0.24]) array([0.]) array([0.]) array([0.])]


In [28]:
X.shape[0]

79314

In [29]:
X.shape[1]

5

In [30]:
X = X.reshape(X.shape[0], 1, X.shape[1])
X[:5]

array([[[0, 0, 0, 0, 0]],

       [[array([0.]), 0, 0, 0, 0]],

       [[array([0.]), array([0.]), 0, 0, 0]],

       [[array([0.]), array([0.]), array([0.]), 0, 0]],

       [[array([0.]), array([0.]), array([0.]), array([0.]), 0]]],
      dtype=object)

In [31]:
X.shape

(79314, 1, 5)

In [32]:
# fit an LSTM network to training data
def fit_lstm(train, batch_size=1, nb_epoch=5, neurons=10):
    # simple early stopping

    #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
    #mc = ModelCheckpoint(filepath="model/model1.h5", save_best_only=True)
  
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1))
    model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
        
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False#, callbacks=[mc,es]
                 )
        model.reset_states()
    model.save("model/model1.h5")
    return model
    #print(model.summary)


In [33]:

# make a one-step forecast
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    #print(X)
    yhat = model.predict(X, batch_size=1)
    return yhat[0,0]

In [34]:
X.shape

(79314, 1, 5)

In [35]:
%time

# transform data to be stationary
raw_values = df.values
diff_values = difference(raw_values, 1)

# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, lag_exp)
supervised_values = supervised.values

# split data into train and test-sets
train, test = supervised_values[0:-predict_values_exp], supervised_values[-predict_values_exp:]


# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)

# fit the model


lstm_model = fit_lstm(train_scaled, batch_size_exp, epoch_exp, neurons_exp)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs
79314/79314 [==============================] - 190s 2ms/step - loss: 0.0035 - root_mean_squared_error: 0.0589


In [36]:
#PLOTS

In [37]:
# walk-forward validation on the test data
predictions = list()
expectations = list()
predictions_plot = list()
expectations_plot = list()
test_pred = list()


In [38]:
test_scaled[:3]

array([[-0.02768549, -0.02768549, -0.02768549, -0.02768549, -0.12070875,
        -0.02768549],
       [-0.02768549, -0.02768549, -0.02768549, -0.02768549, -0.02768549,
        -0.02768549],
       [-0.02768549, -0.02768549, -0.02768549, -0.02768549, -0.02768549,
         0.09634551]])

In [39]:
test_scaled[1, 0:-1]

array([-0.02768549, -0.02768549, -0.02768549, -0.02768549, -0.02768549])

In [40]:
test_scaled[1, -1]

-0.027685492801771905

In [41]:
lstm_model

In [42]:
# make a one-step forecast
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    #print(X)
    yhat = model.predict(X, batch_size=1)
    return yhat[0,0]

In [43]:
X=test_scaled[1, 0:-1]
yhat = forecast_lstm(lstm_model, 1, X)
yhat

-0.01982584

In [44]:
test_pred = [yhat] + test_pred 

In [45]:
test_pred

[-0.01982584]

In [46]:
len(test_pred)

1

In [47]:
X

array([-0.02768549, -0.02768549, -0.02768549, -0.02768549, -0.02768549])

In [48]:
yhat = forecast_lstm(lstm_model, 1, X)
yhat

-0.025433537

In [49]:
raw_values

array([[23.28],
       [23.28],
       [23.28],
       ...,
       [20.7 ],
       [21.25],
       [21.25]])

In [50]:
len(test_scaled)

60

In [51]:
yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-2)#i=2
yhat

array([19.18456646])

In [52]:
len(train)

79314

In [53]:
raw_values

array([[23.28],
       [23.28],
       [23.28],
       ...,
       [20.7 ],
       [21.25],
       [21.25]])

In [54]:
expected = raw_values[len(train) + 2 + 1] #i=2
expected

array([19.77])

In [55]:
lag_exp=5

for i in range(len(test_scaled)):
    # make one-step forecast
    X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
    yhat = forecast_lstm(lstm_model, 1, X)#batch_size_exp to 1

    # Replacing value in test scaled with the predicted value.
    test_pred = [yhat] + test_pred 
    if len(test_pred) > lag_exp+1:#block divided into 5
        test_pred = test_pred[:-1]
    if i+1<len(test_scaled):
        if i+1 > lag_exp+1:
            test_scaled[i+1] = test_pred
        else:
            test_scaled[i+1] = np.concatenate((test_pred, test_scaled[i+1, i+1:]),axis=0)

    # invert scaling
    yhat = invert_scale(scaler, X, yhat)
    # invert differencing
    yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
    # store forecast
    expected = raw_values[len(train) + i + 1]
    predictions_plot.append(yhat)
    expectations_plot.append(expected)
    if expected != 0:
        predictions.append(yhat)
        expectations.append(expected)

        
        
    print('Minute=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))


expectations = np.array(expectations)
predictions = np.array(predictions)

print("Mean Absolute Percent Error: ", (np.mean(np.abs((expectations - predictions) / expectations))))


ValueError: could not broadcast input array from shape (7,) into shape (6,)

In [ ]:


# line plot of observed vs predicted

#sns.set_style("whitegrid")
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
plt.figure(figsize=(20,10))
plt.plot(expectations_plot[:predict_values_exp], label="True")
plt.plot(predictions_plot[:predict_values_exp], label="Predicted")
plt.legend(loc='upper right')
plt.xlabel("Number of Minutes")
plt.ylabel("Energy Demand")
plt.savefig('img/Model_predictiond.png')
#plt.show()